In [1]:
import os
import psycopg2
import openpyxl
import importlib
import pandas as pd
import __init__ as ini

In [6]:
hdfcamc_file = os.path.join(ini.data_dir, 'hdfcamc.xlsx')
pnl = pd.read_excel(hdfcamc_file, sheet_name="profit_loss")
pnl.rename(columns={pnl.columns[0]: 'Date'}, inplace=True)

In [12]:
pnl

,Unnamed: 0,2014-03-01 00:00:00,2015-03-01 00:00:00,2016-03-01 00:00:00,2017-03-01 00:00:00,2018-03-01 00:00:00,2019-03-01 00:00:00,2020-03-01 00:00:00,2021-03-01 00:00:00,2022-03-01 00:00:00,2023-03-01 00:00:00,2024-03-01 00:00:00,2025-03-01 00:00:00
0,Sales -,898.0000,1056.0000,1490.0000,1568.0000,1870.0000,2075.0000,2124.0000,2194.0000,2429.0000,2478.0000,3159.0000,4050.0000
1,Sales Growth %,0.1554,0.1762,0.4116,0.0524,0.1923,0.1097,0.0239,0.0327,0.1070,0.0204,0.2746,0.2820
2,Expenses -,372.0000,432.0000,775.0000,776.0000,802.0000,709.0000,431.0000,388.0000,515.0000,549.0000,623.0000,704.0000
3,Manufacturing Cost %,0.0328,0.0289,0.0230,0.0242,0.1882,0.1296,0.0205,0.0158,0.0214,0.0218,0.0201,0.0193
4,Employee Cost %,0.1253,0.1220,0.0971,0.1014,0.1005,0.1002,0.1028,0.1050,0.1299,0.1274,0.1130,0.0967
5,Other Cost %,0.2566,0.2579,0.4000,0.3693,0.1403,0.1121,0.0796,0.0563,0.0609,0.0723,0.0641,0.0578
6,Operating Profit,525.0000,624.0000,715.0000,792.0000,1068.0000,1366.0000,1693.0000,1806.0000,1913.0000,1929.0000,2536.0000,3346.0000
7,OPM %,0.5900,0.5900,0.4800,0.5100,0.5700,0.6600,0.8000,0.8200,0.7900,0.7800,0.8000,0.8300
8,Other Income -,6.0000,9.0000,4.0000,20.0000,0.0000,22.0000,19.0000,8.0000,4.0000,4.0000,4.0000,9.0000
9,Exceptional items,0.0000,0.0000,0.0000,0.0000,0.0700,-0.2000,0.0400,0.0400,0.0300,0.1100,0.0500,0.0800


In [7]:
pnlt = pnl.transpose() # Transpose the Table for horizontal format
pnlt.columns = pnlt.iloc[0]  # The column name i.e. the mertices are in the first row
pnlt = pnlt.iloc[1:]  # Dropping the first row

pnlt = pnlt.reset_index() # Date is in the index
# pnlt.rename(columns={pnlt.columns[0]: 'Date'}, inplace=True)  # renaming the new columna as Date

# The date is fetched wrongly from the excel sheet as start of the month instead of end of the month
pnlt['Date'] = pnlt['Date'] + pd.offsets.MonthEnd(0) # Setting 0 as parameter ensures that the date does not move to next month incase the date already is end of month

In [8]:
pnlt.index.name = None  # Remove the name

In [10]:
pnlt.reset_index(drop=True, inplace=True)  # Reset to default RangeIndex

In [11]:
pnlt

Unnamed: 0,Date,Sales -,Sales Growth %,Expenses -,Manufacturing Cost %,Employee Cost %,Other Cost %,Operating Profit,OPM %,Other Income -,...,Depreciation,Profit before tax,Tax %,Net Profit -,Exceptional items AT,Profit excl Excep,Profit for PE,Profit for EPS,EPS in Rs,Dividend Payout %
0,2014-03-31,898.0,0.1554,372.0,0.0328,0.1253,0.2566,525.0,0.59,6.0,...,8.0,522.0,0.32,358.0,0.0,358.0,358.0,358.0,141.74,0.35
1,2015-03-31,1056.0,0.1762,432.0,0.0289,0.122,0.2579,624.0,0.59,9.0,...,10.0,623.0,0.33,416.0,0.0,416.0,416.0,416.0,164.61,0.39
2,2016-03-31,1490.0,0.4116,775.0,0.023,0.0971,0.4,715.0,0.48,4.0,...,11.0,708.0,0.33,478.0,0.0,478.0,478.0,478.0,189.9,0.42
3,2017-03-31,1568.0,0.0524,776.0,0.0242,0.1014,0.3693,792.0,0.51,20.0,...,12.0,800.0,0.31,550.0,0.0,550.0,550.0,550.0,218.64,0.42
4,2018-03-31,1870.0,0.1923,802.0,0.1882,0.1005,0.1403,1068.0,0.57,0.0,...,9.0,1058.0,0.33,711.0,0.0,711.0,711.0,711.0,33.78,0.47
5,2019-03-31,2075.0,0.1097,709.0,0.1296,0.1002,0.1121,1366.0,0.66,22.0,...,13.0,1375.0,0.32,931.0,0.0,931.0,931.0,931.0,43.78,0.55
6,2020-03-31,2124.0,0.0239,431.0,0.0205,0.1028,0.0796,1693.0,0.8,19.0,...,50.0,1653.0,0.24,1262.0,0.0,1262.0,1262.0,1262.0,59.32,0.47
7,2021-03-31,2194.0,0.0327,388.0,0.0158,0.105,0.0563,1806.0,0.82,8.0,...,55.0,1749.0,0.24,1326.0,0.0,1326.0,1326.0,1326.0,62.26,0.55
8,2022-03-31,2429.0,0.107,515.0,0.0214,0.1299,0.0609,1913.0,0.79,4.0,...,54.0,1855.0,0.25,1393.0,0.0,1393.0,1393.0,1393.0,65.32,0.64
9,2023-03-31,2478.0,0.0204,549.0,0.0218,0.1274,0.0723,1929.0,0.78,4.0,...,53.0,1871.0,0.24,1424.0,0.0,1424.0,1424.0,1424.0,66.72,0.72
